# Technical Factor - Price Oscillator (PRICEOSC)

In [16]:
import dai
import pandas as pd

In [17]:
sd = '2025-01-01'
ed = '2026-01-01'

instrument_list = ['600519.SH']

In [18]:
sql = f"""
WITH
data_base AS (
    SELECT
        date,
        instrument,
        open,
        high,
        low,
        close,
        volume,
    FROM cn_stock_bar1d
),

data_factor AS (
    SELECT
        date,
        instrument,
        100 * (m_avg(close, 13) - m_avg(close, 26)) / m_avg(close, 13) AS PRICEOSC,
    FROM data_base
),

data_signal_trend AS (
    SELECT
        date,
        instrument,
        IF(PRICEOSC > 0 AND m_lag(PRICEOSC, 1) <= 0, 1, 0) AS TRBY1,
        IF(PRICEOSC < 0 AND m_lag(PRICEOSC, 1) >= 0, 1, 0) AS TRSL1,
    FROM data_factor
),

data_signal_momentum AS (
    SELECT
        date,
        instrument,
        IF(PRICEOSC > 0 AND (PRICEOSC - m_lag(PRICEOSC, 1)) > 0, 1, 0) AS MTBY1,
        IF(PRICEOSC < 0 AND (PRICEOSC - m_lag(PRICEOSC, 1)) < 0, 1, 0) AS MTSL1,
    FROM data_factor
),

data_signal_reversal AS (
    SELECT
        date,
        instrument,
        IF(PRICEOSC < -2 AND (PRICEOSC - m_lag(PRICEOSC, 1)) > 0, 1, 0) AS RVBY1,
        IF(PRICEOSC > 2  AND (PRICEOSC - m_lag(PRICEOSC, 1)) < 0, 1, 0) AS RVSL1,
    FROM data_factor
),

data_signal_breakout AS (
    SELECT
        date,
        instrument,
        IF(m_lag(PRICEOSC, 1) <= 1  AND PRICEOSC > 1, 1, 0)  AS BKBY1,
        IF(m_lag(PRICEOSC, 1) >= -1 AND PRICEOSC < -1, 1, 0) AS BKSL1,
    FROM data_factor
),

data_combined AS (
    SELECT
        date,
        instrument,
        PRICEOSC,
        TRBY1,
        TRSL1,
        MTBY1,
        MTSL1,
        RVBY1,
        RVSL1,
        BKBY1,
        BKSL1,
    FROM data_factor
    JOIN data_signal_trend    USING (date, instrument)
    JOIN data_signal_momentum USING (date, instrument)
    JOIN data_signal_reversal USING (date, instrument)
    JOIN data_signal_breakout USING (date, instrument)
)

SELECT *
FROM data_combined
QUALIFY COLUMNS(*) IS NOT NULL
ORDER BY date, instrument
"""

In [19]:
df = dai.query(sql, filters={"date":[sd,ed], "instrument":instrument_list}).df()
df

,date,instrument,PRICEOSC,TRBY1,TRSL1,MTBY1,MTSL1,RVBY1,RVSL1,BKBY1,BKSL1
0,2025-02-14,600519.SH,-0.590726,0,0,0,0,0,0,0,0
1,2025-02-17,600519.SH,-0.528302,0,0,0,0,0,0,0,0
2,2025-02-18,600519.SH,-0.345970,0,0,0,0,0,0,0,0
3,2025-02-19,600519.SH,-0.225367,0,0,0,0,0,0,0,0
4,2025-02-20,600519.SH,-0.112867,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...
213,2025-12-25,600519.SH,-0.683449,0,0,0,0,0,0,0,0
214,2025-12-26,600519.SH,-0.403592,0,0,0,0,0,0,0,0
215,2025-12-29,600519.SH,-0.169180,0,0,0,0,0,0,0,0
216,2025-12-30,600519.SH,-0.057691,0,0,0,0,0,0,0,0
